SABINA COMMENT: What about my notebook sections? :(

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

In [10]:
RAW_DIR = Path.cwd()  
RAW_DIR

WindowsPath('c:/Users/SabinaFirtala/Desktop/projects/ironhack/cohort/october-25/vanguard-project/Vanguard-AB-testing-project-')

In [11]:
list(RAW_DIR.glob("df_final_*.txt"))
# confirm that the files are in the directory

[WindowsPath('c:/Users/SabinaFirtala/Desktop/projects/ironhack/cohort/october-25/vanguard-project/Vanguard-AB-testing-project-/df_final_demo.txt'),
 WindowsPath('c:/Users/SabinaFirtala/Desktop/projects/ironhack/cohort/october-25/vanguard-project/Vanguard-AB-testing-project-/df_final_experiment_clients.txt')]

In [5]:
OUT_DIR = RAW_DIR / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR
# confirm that the processed directory is created

PosixPath('/Users/marisa/Desktop/Data Analytics/Week11/Vanguard-AB-testing/processed')

In [ ]:
pd.read_csv("data/raw/df_final_demo.txt")
pd.read_csv("data/raw/df_final_experiment_clients.txt")
pd.read_csv("data/raw/df_final_web_data_pt_1.txt")
pd.read_csv("data/raw/df_final_web_data_pt_2.txt")

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [ ]:
def read_txt_any_sep(path: Path) -> pd.DataFrame:
    # SABINA COMMENT: This function is not necessary from what I've tested (code cell above)
    # Did you have issues in Mac with this?
    for sep in [",", "|", "\t", ";"]:
        try:
            df = pd.read_csv(path, sep=sep)
            if df.shape[1] > 1:
                return df
        except Exception:
            pass
    return pd.read_csv(path, sep=None, engine="python")

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(" ", "_")
          .str.replace("-", "_")
    )
    return df
    # function to clean column names


In [8]:
demo_path = RAW_DIR / "df_final_demo.txt"
exp_path  = RAW_DIR / "df_final_experiment_clients.txt"
web1_path = RAW_DIR / "df_final_web_data_pt_1.txt"
web2_path = RAW_DIR / "df_final_web_data_pt_2.txt"

df_demo = clean_cols(read_txt_any_sep(demo_path))
df_exp  = clean_cols(read_txt_any_sep(exp_path))
df_web  = pd.concat(
    [clean_cols(read_txt_any_sep(web1_path)),
     clean_cols(read_txt_any_sep(web2_path))],
    ignore_index=True
)

df_demo.shape, df_exp.shape, df_web.shape
# check the shapes of the dataframes

((70609, 9), (70609, 2), (755405, 5))

In [ ]:
df_web["date_time"] = pd.to_datetime(df_web["date_time"], errors="coerce")

df_web = df_web.dropna(subset=["date_time", "process_step", "visit_id", "client_id"])
# SABINA COMMENT: Careful about the one below! What if a client repeats a process_step?
df_web = df_web.drop_duplicates(subset=["client_id", "visit_id", "process_step", "date_time"])

df_events = df_web.merge(df_exp[["client_id", "variation"]], on="client_id", how="inner")

df_events["variation"] = df_events["variation"].astype(str).str.strip().str.title()

# SABINA COMMENT: Why use .copy() when you're saving this in the same dataframe?
df_events = df_events[df_events["variation"].isin(["Control", "Test"])].copy()

df_events["variation"].value_counts()
# check the variation values after cleaning

variation
Test       176699
Control    140536
Name: count, dtype: int64

In [ ]:
# SABINA COMMENT: Don't think this necessary but it's very thorough of you haha.
# Saying this because even if the A/B test lasted slightly longer, that's okay, just a few more rows to look at.
# But if you have a deeper reason for doing this, I'm all ears.
start = pd.Timestamp("2017-03-15")
end   = pd.Timestamp("2017-06-20 23:59:59")
df_events = df_events[(df_events["date_time"] >= start) & (df_events["date_time"] <= end)].copy()

df_events["date_time"].min(), df_events["date_time"].max(), df_events.shape
# check the date_time range and shape after filtering

(Timestamp('2017-03-15 00:19:28'),
 Timestamp('2017-06-20 23:57:06'),
 (317235, 6))

In [11]:
sorted(df_events["process_step"].astype(str).str.lower().unique())
# check unique process_step values after cleaning

['confirm', 'start', 'step_1', 'step_2', 'step_3']

In [ ]:
df_events["process_step"] = df_events["process_step"].astype(str).str.strip().str.lower()

aliases = {
    "landing": "start",
    "initial": "start",
    "confirmation": "confirm",
}

df_events["process_step"] = df_events["process_step"].replace(aliases)

step_order = ["start", "step_1", "step_2", "step_3", "confirm"]
step_rank = {s: i for i, s in enumerate(step_order)}

# SABINA COMMENT: Check out pd.Categorical for another way of doing this :)
df_events = df_events[df_events["process_step"].isin(step_rank)].copy()
df_events["step_rank"] = df_events["process_step"].map(step_rank)

# SABINA COMMENT: I wouldn't do this - in the off-case where you have 2 steps happening in the same second (highly unlikely)
# you want the order originally recorded by the system since people can backtrack.
df_events = df_events.sort_values(["visit_id", "date_time", "step_rank"]).reset_index(drop=True)

df_events[["visit_id","process_step","date_time","step_rank"]].head(10)
# display the first 10 rows of relevant columns after processing

,visit_id,process_step,date_time,step_rank
0,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,4
1,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,4
2,100019538_17884295066_43909,start,2017-04-09 16:20:56,0
3,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,1
4,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,2
5,100019538_17884295066_43909,step_1,2017-04-09 16:21:35,1
6,100019538_17884295066_43909,step_1,2017-04-09 16:21:41,1
7,100019538_17884295066_43909,start,2017-04-09 16:21:45,0
8,100019538_17884295066_43909,start,2017-04-09 16:21:59,0
9,100019538_17884295066_43909,step_1,2017-04-09 16:22:04,1


In [ ]:
events = df_events.copy()

events["next_step_rank"] = events.groupby("visit_id")["step_rank"].shift(-1)
events["next_time"]      = events.groupby("visit_id")["date_time"].shift(-1)

events["delta_sec"] = (events["next_time"] - events["date_time"]).dt.total_seconds()

step_durations = events.dropna(subset=["next_step_rank", "delta_sec"]).copy()
step_durations["is_backtrack"] = step_durations["next_step_rank"] < step_durations["step_rank"]

# remove negative / super long idle (30min cap)
# SABINA COMMENT: Do you have any negatives??? This shouldn't be possible xD
# Nice re: removing idles :)
step_durations = step_durations[(step_durations["delta_sec"] >= 0) & (step_durations["delta_sec"] <= 1800)].copy()

step_durations = step_durations[[
    "client_id","visit_id","variation",
    "process_step","step_rank","next_step_rank","delta_sec","is_backtrack"
]].copy()

step_durations.head()
# display the first few rows of step durations with backtrack info

,client_id,visit_id,variation,process_step,step_rank,next_step_rank,delta_sec,is_backtrack
0,3561384,100012776_37918976071_457913,Test,confirm,4,4.0,52.0,False
2,7338123,100019538_17884295066_43909,Test,start,0,1.0,16.0,False
3,7338123,100019538_17884295066_43909,Test,step_1,1,2.0,9.0,False
4,7338123,100019538_17884295066_43909,Test,step_2,2,1.0,14.0,True
5,7338123,100019538_17884295066_43909,Test,step_1,1,1.0,6.0,False


In [ ]:
# SABINA COMMENT: Ah, all of this code is so clean and clear. :heart-eyes:
# Also learnt 2 new tricks from you, thanks!
confirm_rank = step_rank["confirm"]

sessions = (
    df_events.groupby(["visit_id","client_id","variation"], as_index=False)
      .agg(
          n_events=("process_step","size"),
          first_time=("date_time","min"),
          last_time=("date_time","max"),
          max_step=("step_rank","max"),
      )
)

sessions["completed"] = sessions["max_step"].ge(confirm_rank)
sessions["session_duration_sec"] = (sessions["last_time"] - sessions["first_time"]).dt.total_seconds()

bt = step_durations.groupby("visit_id", as_index=False)["is_backtrack"].sum().rename(columns={"is_backtrack":"num_backtracks"})
sessions = sessions.merge(bt, on="visit_id", how="left")
sessions["num_backtracks"] = sessions["num_backtracks"].fillna(0).astype(int)

# SABINA COMMENT: really like this, nice work.
sessions.head()
# display the first few rows of session summarys with completion and backtrack info

,visit_id,client_id,variation,n_events,first_time,last_time,max_step,completed,session_duration_sec,num_backtracks
0,100012776_37918976071_457913,3561384,Test,2,2017-04-26 13:22:17,2017-04-26 13:23:09,4,True,52.0,0
1,100019538_17884295066_43909,7338123,Test,11,2017-04-09 16:20:56,2017-04-09 16:24:58,4,True,242.0,2
2,100022086_87870757897_149620,2478628,Test,5,2017-05-23 20:44:01,2017-05-23 20:47:01,4,True,180.0,0
3,100030127_47967100085_936361,105007,Control,1,2017-03-22 11:07:49,2017-03-22 11:07:49,0,False,0.0,0
4,100037962_47432393712_705583,5623007,Control,4,2017-04-14 16:41:51,2017-04-14 16:44:03,1,False,132.0,1


In [ ]:
# SABINA COMMENT: I can't even tell you just how much I enjoyed reading this code and
# table below. Chapeau! :hat:
kpi_completion = (
    sessions.groupby("variation")
      .agg(
          n_sessions=("visit_id","nunique"),
          n_completed=("completed","sum"),
          completion_rate=("completed","mean"),
          median_session_duration_sec=("session_duration_sec","median"),
          mean_session_duration_sec=("session_duration_sec","mean"),
          avg_backtracks=("num_backtracks","mean"),
      )
      .reset_index()
)

kpi_errors = (
    step_durations.groupby("variation")
      .agg(n_transitions=("delta_sec","size"), 
           n_backtracks=("is_backtrack","sum"))
      .reset_index()
)
kpi_errors["error_rate"] = kpi_errors["n_backtracks"] / kpi_errors["n_transitions"]

kpi_step_time = (
    step_durations[~step_durations["is_backtrack"]]
      .groupby(["variation","process_step"])["delta_sec"]
      .mean()
      .reset_index()
      .rename(columns={"delta_sec":"avg_time_to_next_step_sec"})
)

kpis_by_variation = kpi_completion.merge(kpi_errors, on="variation", how="left")
kpis_by_variation
# display the KPIs by variation

,variation,n_sessions,n_completed,completion_rate,median_session_duration_sec,mean_session_duration_sec,avg_backtracks,n_transitions,n_backtracks,error_rate
0,Control,32189,16088,0.498961,160.0,280.510653,0.300965,108254,9513,0.087877
1,Test,37136,21791,0.585717,168.0,315.455999,0.441297,139422,16229,0.116402


In [16]:
inv_rank = {v:k for k,v in step_rank.items()}

funnel = (
    sessions.groupby(["variation","max_step"])
      .size()
      .reset_index(name="n_sessions")
)
funnel["max_step_label"] = funnel["max_step"].map(inv_rank)
funnel.sort_values(["variation","max_step"])
# display the funnel by variation and max step reached

,variation,max_step,n_sessions,max_step_label
0,Control,0,7572,start
1,Control,1,3494,step_1
2,Control,2,2040,step_2
3,Control,3,3049,step_3
4,Control,4,16088,confirm
5,Test,0,5082,start
6,Test,1,3908,step_1
7,Test,2,2573,step_2
8,Test,3,3850,step_3
9,Test,4,21791,confirm


In [17]:
df_events.to_csv(OUT_DIR / "events.csv", index=False)
sessions.to_csv(OUT_DIR / "sessions.csv", index=False)
step_durations.to_csv(OUT_DIR / "step_durations.csv", index=False)
kpis_by_variation.to_csv(OUT_DIR / "kpis_by_variation.csv", index=False)
kpi_step_time.to_csv(OUT_DIR / "kpi_step_time.csv", index=False)
funnel.to_csv(OUT_DIR / "funnel_max_step.csv", index=False)

print("Saved to:", OUT_DIR)
list(OUT_DIR.glob("*.csv"))
# save processed data to CSV files

Saved to: /Users/marisa/Desktop/Data Analytics/Week11/Vanguard-AB-testing/processed


[PosixPath('/Users/marisa/Desktop/Data Analytics/Week11/Vanguard-AB-testing/processed/step_durations.csv'),
 PosixPath('/Users/marisa/Desktop/Data Analytics/Week11/Vanguard-AB-testing/processed/kpis_by_variation.csv'),
 PosixPath('/Users/marisa/Desktop/Data Analytics/Week11/Vanguard-AB-testing/processed/kpi_step_time.csv'),
 PosixPath('/Users/marisa/Desktop/Data Analytics/Week11/Vanguard-AB-testing/processed/events.csv'),
 PosixPath('/Users/marisa/Desktop/Data Analytics/Week11/Vanguard-AB-testing/processed/funnel_max_step.csv'),
 PosixPath('/Users/marisa/Desktop/Data Analytics/Week11/Vanguard-AB-testing/processed/sessions.csv')]

SABINA COMMENT: The analysis more or less speaks for itself, what a pleasure to see. Would just write some thoughts on the last table, otherwise you're good. Well done!